<a href="https://colab.research.google.com/github/ianomunga/backTrack/blob/main/rails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install nltk

In [22]:
import pandas as pd

tweets_df = pd.read_csv('/content/health_tweets.csv')
tweets_df.dropna(axis='columns', inplace=True)
tweets_df.columns
tweets_df = tweets_df[['date', 'timezone', 'tweet', 
                       'hashtags', 'username', 'name', 
                       'day', 'hour', 'retweet', 'nlikes', 
                       'nreplies', 'nretweets']]
tweets_df.drop_duplicates(inplace=True, subset="tweet")

import re
import gensim
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'       
# define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub('📝 …', '', tweet)
    return tweet

def tokenize_tweets(df):
    """Main function to read in and return cleaned and preprocessed dataframe.
    This can be used in Jupyter notebooks by importing this module and calling the tokenize_tweets() function
    Args:
        df = data frame object to apply cleaning to
    Returns:
        pandas data frame with cleaned tokens
    """

    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

tweets_df = tokenize_tweets(tweets_df)
tweets_df.head(69)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Complete. Number of Tweets that have been cleaned and tokenized : 197925


,date,timezone,tweet,hashtags,username,name,day,hour,retweet,nlikes,nreplies,nretweets,tokens
0,2020-03-28 00:10:53,UTC,How the UK’s coronavirus epidemic compares to ...,[],bbchealth,BBC Health News,2,13,False,20,4,9,coronavirus epidemic compare countries
1,2020-03-27 17:46:51,UTC,Health workers on frontline to be tested in En...,[],bbchealth,BBC Health News,1,20,False,34,5,15,health workers frontline test england
2,2020-03-27 06:02:12,UTC,Coronavirus: Protective gear guidance 'to be u...,[],bbchealth,BBC Health News,2,12,False,24,4,15,coronavirus protective gear guidance update
3,2020-03-27 02:03:35,UTC,Coronavirus: What are ventilators and why are ...,[],bbchealth,BBC Health News,4,19,False,30,1,24,coronavirus ventilators important
4,2020-03-26 19:58:34,UTC,Coronavirus: 'Act early to save more than 30 m...,[],bbchealth,BBC Health News,3,8,False,6,0,3,coronavirus act early save million live
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2020-02-19 02:26:05,UTC,"Children facing uncertain future, experts warn...",[],bbchealth,BBC Health News,1,10,False,21,2,16,children face uncertain future experts warn
65,2020-02-18 17:14:12,UTC,Heart doctors 'held back stent death data' htt...,[],bbchealth,BBC Health News,3,4,False,42,6,33,heart doctor hold stent death data
66,2020-02-18 00:59:12,UTC,Criminals end up with 'smaller brains' http://...,[],bbchealth,BBC Health News,5,2,False,12,1,14,criminals end smaller brain
67,2020-02-18 00:03:55,UTC,Transgender patients self-medicating over NHS ...,[],bbchealth,BBC Health News,1,17,False,7,0,5,transgender patients self medicate nhs wait
